In [1]:
import os
import json
import warnings
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from peft import PeftModel, PeftConfig

import bentoml


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jake/codes/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Loading binary /home/jake/codes/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jake/codes/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/jake/codes/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('unix')}
  warn(msg)
/home/jake/codes/news-topic-keyphrase-generation-model-ops/.venv/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('ELECTRON_RUN_AS_NODE/w'), PosixPath('WT_PROFILE_ID\n'), PosixPath('WT_SESSION')}
  warn(msg)
/home/jake/codes/news-topic-keyphras

# Save bento model

In [2]:
# CKPT = "checkpoints/paust_pko_t5_base_v4_run_18/merged"
# model = AutoModelForSeq2SeqLM.from_pretrained(CKPT)

In [3]:
# bento_model = bentoml.pytorch.save_model(
#     "news-topic-keyphrase-generator-t5-base",
#     model,
#     signatures={
#         "generate": {"batchable": False},
#     },
# )
# print(bento_model)

# Test bento model

In [4]:
# load bento model

runner = bentoml.pytorch.get("news-topic-keyphrase-generator-t5-base:latest").to_runner()

In [5]:
runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [6]:
tokenizer_path = "checkpoints/paust_pko_t5_base_v4_run_18"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [7]:
prefix = "generate keyphrases: "

def news_topic_keyphrase_generator_t5_base(input_texts_json):
    inputs = [prefix + input_ for input_ in input_texts_json]
    encoded = tokenizer(
        inputs,
        max_length=1024,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    outputs = runner.generate.run(**encoded, max_length=64).cpu()
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return outputs

In [8]:
test_case = """'결정만 남은' 맨유, 김민재 바이아웃 발동 고려 중 [포포투=백현기]맨체스터 유나이티드는 김민재 영입에 진심이다.나폴리는 리그 우승을 눈앞에 두고 있다. 나폴리는 이번 시즌 스쿠데토에 가까워져 있다. 현재 리그 28경기를 치른 시점에서 승점 71점으로 선두를 기록하고 있다. 나폴리는 디에고 마라도나가 군림했던 1989-90시즌 이후 첫 우승이 임박했다. 또한 나폴리는 유럽축구연맹(UEFA) 챔피언스리그(UCL) 8강에 진출해 있으며 구단 역사상 최고 성적을 거두고 있다.나폴리의 고공 행진의 중심에 수비수 김민재가 있다. 김민재는 이번 시즌 '역대급 시즌'을 보내고 있다. 데뷔 시즌이라는 사실이 무색할 만큼 세리에 A에서의 첫 시즌에서 안정적인 수비력을 보여주고 있다. 특히 김민재의 안정적인 수비력과 넓은 커버 범위 덕분에 나폴리는 마음 놓고 공격 라인을 더 높이 올릴 수 있다.첫 시즌부터 세리에 A에 연착륙해 맹활약하고 있는 김민재에게 극찬도 쏟아졌다. 지난 1월에는 AS 로마의 조세 무리뉴 감독이 "나폴리 선수 중 한 명에 대해 얘기할 말이 있다. 과거 토트넘에 있을 당시 김민재와 화상 통화도 했을 정도로 그를 영입하고 싶었다. 하지만 토트넘은 나를 도와주지 않았다"라고 김민재를 극찬했다.이어 루치아노 스팔레티 감독도 김민재를 아끼고 있다. 김민재의 수비력과 커버 범위, 축구 지능을 믿고 나폴리는 더 공격적인 축구 구현이 가능해졌다. 스팔레티 감독은 핵심이 된 김민재에게 "그는 내게 있어 세계 최고의 수비수"라는 극찬을 쏟아내기도 했다.자연스럽게 빅클럽들의 관심을 받고 있다. 가장 적극적인 구단은 맨체스터 유나이티드다. 유럽축구 이적시장 전문가 파브리시오 로마노에 따르면, 맨체스터 유나이티드는 김민재를 관찰하기 위해 이미 스카우터를 파견했으며 꾸준하게 관찰 중이라 전해진다.김민재의 이적의 가장 큰 변수는 바로 상대적으로 낮은 바이아웃 금액이다. 영국 매체 '데일리 메일'에 따르면, 김민재는 올 여름 일시적으로 4500만 유로(약 646억 원)의 바이아웃 조항을 가지고 있다. 김민재의 시장 가치에 비해 상대적으로 낮은 바이아웃 금액에 나폴리는 김민재와의 새 계약을 시즌 내내 추진하고 있다.맨유는 이 바이아웃에 집중하고 있다. '데일리 메일'은 "맨유는 김민재의 바이아웃 조항을 발동하는 것을 고려 중이다"고 밝혔다. 매체는 이어 "리버풀과 토트넘 훗스퍼 또한 김민재 영입을 위해 경쟁하고 있다"고 덧붙이면서 잉글랜드 클럽들의 관심을 소개하기도 했다.기사제공 포포투백현기 기자 hkbaek11@fourfourtwo.co.krCopyright c 포포투. All rights reserved. 무단 전재 및 재배포 금지.기사 섹션 분류 가이드포포투 FourFourTwo 잡지 정기 구독하기"""

input_texts = [test_case]

print(input_texts)

['\'결정만 남은\' 맨유, 김민재 바이아웃 발동 고려 중 [포포투=백현기]맨체스터 유나이티드는 김민재 영입에 진심이다.나폴리는 리그 우승을 눈앞에 두고 있다. 나폴리는 이번 시즌 스쿠데토에 가까워져 있다. 현재 리그 28경기를 치른 시점에서 승점 71점으로 선두를 기록하고 있다. 나폴리는 디에고 마라도나가 군림했던 1989-90시즌 이후 첫 우승이 임박했다. 또한 나폴리는 유럽축구연맹(UEFA) 챔피언스리그(UCL) 8강에 진출해 있으며 구단 역사상 최고 성적을 거두고 있다.나폴리의 고공 행진의 중심에 수비수 김민재가 있다. 김민재는 이번 시즌 \'역대급 시즌\'을 보내고 있다. 데뷔 시즌이라는 사실이 무색할 만큼 세리에 A에서의 첫 시즌에서 안정적인 수비력을 보여주고 있다. 특히 김민재의 안정적인 수비력과 넓은 커버 범위 덕분에 나폴리는 마음 놓고 공격 라인을 더 높이 올릴 수 있다.첫 시즌부터 세리에 A에 연착륙해 맹활약하고 있는 김민재에게 극찬도 쏟아졌다. 지난 1월에는 AS 로마의 조세 무리뉴 감독이 "나폴리 선수 중 한 명에 대해 얘기할 말이 있다. 과거 토트넘에 있을 당시 김민재와 화상 통화도 했을 정도로 그를 영입하고 싶었다. 하지만 토트넘은 나를 도와주지 않았다"라고 김민재를 극찬했다.이어 루치아노 스팔레티 감독도 김민재를 아끼고 있다. 김민재의 수비력과 커버 범위, 축구 지능을 믿고 나폴리는 더 공격적인 축구 구현이 가능해졌다. 스팔레티 감독은 핵심이 된 김민재에게 "그는 내게 있어 세계 최고의 수비수"라는 극찬을 쏟아내기도 했다.자연스럽게 빅클럽들의 관심을 받고 있다. 가장 적극적인 구단은 맨체스터 유나이티드다. 유럽축구 이적시장 전문가 파브리시오 로마노에 따르면, 맨체스터 유나이티드는 김민재를 관찰하기 위해 이미 스카우터를 파견했으며 꾸준하게 관찰 중이라 전해진다.김민재의 이적의 가장 큰 변수는 바로 상대적으로 낮은 바이아웃 금액이다. 영국 매체 \'데일리 메일\'에 따르면, 김민재는 올 여름 일시적으로 4500만 유로(약 646억 원)

In [9]:
outputs = news_topic_keyphrase_generator_t5_base(input_texts)
outputs

['맨체스터 유나이티드; 김민재; 바이아웃; 세리에 A; 수비수; 세리에 A 연착륙; 수비력; 커버 범위; 빅클럽들; 이적시장']